### COMP3314 ML
Guo Shunhua 3035447635 
<br>
# <u>Logistic regression Construction</u>

## Overview
1. [Introduction](#s1) 
2. [Decision Tree](#s2)
3. [Random Forest](#s3)
4. [Load data & Apply the model](#s4) 
5. [Plotting the result for analysis](#s5)
6. [Parameter Analysis](#s6)


----- 


<a id=’s1’></a>

## 1 Introduction

This notebook will implement a Multi-class Decision Tree, Random Forest, and then train implemented model to provided data set.

<a id=’s2’></a>

## 2 Decision Tree

<u>Import Libraries</u>

In [285]:
import sys
if sys.version_info[0] < 3:
    raise Exception("Python 3 not detected")
                    
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import statistics
import random

As stated in the lecture, the following impurity measures or splitting criteria are commonly used in decision trees:
1. Gini impurity ( gini ), 
2. Entropy ( entropy ), 
3. Classification error ( classificationError ).

In [286]:
# define gini value for one group (classes)
def gini(group, numClass):
    gini = 1  
    size = len(group)
    
    if size != 0:
        y_v = [y[-1] for y in group]
        for c in range(numClass):
            #proportion of each class
            p = y_v.count(c)/size
            gini -= p**2

    return gini 
print(gini([[1, 1], [1, 0]], 2), gini([[1, 0], [1, 0]], 2))

0.5 0.0


In [287]:
# define entropy value for one group (classes)
def entropy(group, numClass):
    entropy = 0  
    size = len(group)
    
    if size != 0:
        y_v = [y[-1] for y in group]
        for c in range(numClass):
            #proportion of each class
            p = y_v.count(c)/size
            entropy -= p * math.log(p,2)

    return entropy 
print(entropy([[1, 1], [1, 0]], 2), gini([[1, 0], [1, 0]], 2))

1.0 0.0


In [288]:
# define classificationError value for one group (classes)
def classificationError(group, numClass):
    p = []
    size = len(group)
    
    if size != 0:
        y_v = [y[-1] for y in group]
        for c in range(numClass):
            #proportion of each class
            p.append(y_v.count(c)/size)

    e = 1 - np.max(p)     
    return e 
print(classificationError([[1, 1], [1, 0]], 2), gini([[1, 0], [1, 0]], 2)) 

0.5 0.0


In [283]:
# construct a split
def split(group, x_index, val):
    #record group spliting result
    left = []
    right = []
    for x in group:
        if x[x_index] < val:
            left.append(x)
        else:
            right.append(x)
    return left, right

In [210]:
# compute information gain at a split point
def info_gain(group, x_index, val, numClass):
    size = len(group)
    #parent's loss
    p = gini(group, numClass)
    
    left, right = split(group, x_index, val)
    
    # proportional left/right group's loss
    l = gini(left, numClass) * len(left) / size
    r = gini(right, numClass) * len(right) / size
    
    return p-l-r

In [211]:
# select split
def select_split(group, numClass):
    info_gain_v = []
    info_gain_comb = []
    for x_index in range(len(group[0])-1):
        if len(group) > 100:   
            x = [x[x_index] for x in group]
            xmin, xmax = np.min(x), np.max(x)
            step_size = (xmax - xmin)/20
            val = xmin + step_size
            while val < xmax:
                ig = info_gain(group, x_index, val, numClass)
                #print("x_index: %f; val: %f; ig: %f" % (x_index, val, ig))

                info_gain_v.append(ig)
                info_gain_comb.append([x_index, val])
                val += step_size
        
        else:
            for x in group:
                ig = info_gain(group, x_index, x[x_index], numClass)
                #print("x_index: %f; val: %f; ig: %f" % (x_index, x[x_index], ig))

                info_gain_v.append(ig)
                info_gain_comb.append([x_index, x[x_index]])

    # get the split resulting maximum infomation gain
    x_index, val = info_gain_comb[np.argmax(info_gain_v)]
    print("FINAL: x_index: %f; val: %f" % (x_index, val))
    return split(group, x_index, val)

In [212]:
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
select_split(dataset, 2)

FINAL: x_index: 0.000000; val: 6.642287


([[2.771244718, 1.784783929, 0],
  [1.728571309, 1.169761413, 0],
  [3.678319846, 2.81281357, 0],
  [3.961043357, 2.61995032, 0],
  [2.999208922, 2.209014212, 0]],
 [[7.497545867, 3.162953546, 1],
  [9.00220326, 3.339047188, 1],
  [7.444542326, 0.476683375, 1],
  [10.12493903, 3.234550982, 1],
  [6.642287351, 3.319983761, 1]])

In [213]:
# Load raw data from csv file, which located in the same folder as this notebook.

X_train_raw = pd.read_csv("dataset_files/car_X_train.csv").to_numpy()
y_train_raw = pd.read_csv("dataset_files/car_y_train.csv").to_numpy()
X_test_raw = pd.read_csv("dataset_files/car_X_test.csv").to_numpy()
y_test_raw = pd.read_csv("dataset_files/car_y_test.csv").to_numpy()

print("X input Meaning: ",
      "buying price, price of the maintenance, number of doors, ",
      "number of persons to carry, size of luggage boot, safety of the car")

print("X training Class List: ", set(X_train_raw.flatten()))
print("y training Class List: ", set(y_train_raw.flatten()))
print("X test Class List: ", set(X_test_raw.flatten()))
print("y test Class List: ", set(y_test_raw.flatten()))

X input Meaning:  buying price, price of the maintenance, number of doors,  number of persons to carry, size of luggage boot, safety of the car
X training Class List:  {'high', 'low', '3', 'small', 'med', 'vhigh', 'more', 'big', '5more', '2', '4'}
y training Class List:  {'acc', 'good', 'unacc', 'vgood'}
X test Class List:  {'high', 'low', 'small', '3', 'med', 'vhigh', 'more', 'big', '5more', '2', '4'}
y test Class List:  {'acc', 'good', 'unacc', 'vgood'}


In [214]:
# transform str indicators in data to numeric classes
x_dict = {
    'vhigh' : 4, 
    'high' : 3,
    'med' : 2,
    'low' : 1, 
     
    'small' : 1,
    # 'med' : 2,
    'big' : 3,
    
    # '2' : 2, 
    # '4' : 4,
    'more' : 6, 
    
    '2' : 2, 
    '3' : 3,
    '4' : 4,
    '5more' : 6,
}

y_dict = {
    'acc':1, 'good':2, 'unacc':0, 'vgood':3
}

def transform_X(X):
    for i in range(len(X)):
        x = X[i]
        X[i] = [x_dict[i] for i in x]
    return X

def transform_y(y):
    return [[y_dict[row[0]]] for row in y]

X_train = transform_X(X_train_raw)
y_train = np.array(transform_y(y_train_raw))
X_test = transform_X(X_test_raw)
y_test = np.array(transform_y(y_test_raw))

In [269]:
class DTnode(object):
    def __init__(self, level=0):
        self.level = level
        
        self.x_index = None
        self.val = None
        self.left = None
        self.right = None
        
    def update_info(self, x_index, val, left, right):
        self.x_index = x_index
        self.val = val
        self.left = left
        self.right = right
        
    def add_class(self, c):
        # print("add c, c = ", c)
        self.c = c
        
    def get_info(self):
        return self.x_index, self.val, self.left, self.right
    

In [270]:
class DecisionTree(object):
    def __init__(self, criterion='gini', max_depth=4, min_size=1):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_size = min_size
    
    # main function for training the model
    def fit(self, X, y):
        self.numClass = int(np.max(y)) + 1
        X_w_y = np.concatenate((X, y), axis = 1)
        
        # gives a group split with max_depth
        # record each node spliting
        self.root = DTnode()
        self.split_node(self.root, X_w_y, self.max_depth)
        
        return self
    
    #recursively split the group
    def split_node(self, node, group, max_depth):
        if len(group) == 0:
            return
        
        if max_depth <= 0 or len(group) <= self.min_size:
            y_v = [y[-1] for y in group]
            c_v = [y_v.count(c) for c in range(self.numClass)]
            c = np.argmax(c_v)
            node.add_class(c)
            return 
        
        (left, right), x_index, val = self.select_split(group)
        leftnode = DTnode(level=node.level+1)
        rightnode = DTnode(level=node.level+1)
        node.update_info(x_index, val, leftnode, rightnode)
        
        self.split_node(leftnode, left, max_depth-1)
        self.split_node(rightnode, right, max_depth-1)
    
    # predict single x input
    def predict(self, node, x):
        x_index, val, leftnode, rightnode = node.get_info()
        
        if x_index == None or val == None:
            return node.c
        elif x[x_index] < val:
            return self.predict(leftnode, x)
        else:
            return self.predict(rightnode, x)
    
    #compute model accuracy
    def accuracy(self, X, y):
        y_pred = [[self.predict(self.root, x)] for x in X]
        return (y_pred == y).sum()/len(y)
            
    # select split
    def select_split(self, group):
        info_gain_v = []
        info_gain_comb = []
        for x_index in range(len(group[0])-1):
            if len(group) > 100:   
                x_v = [x[x_index] for x in group]
                xmin, xmax = np.min(x_v), np.max(x_v)
                step_size = (xmax - xmin)/20
                val = xmin + step_size
                while val < xmax:
                    ig = info_gain(group, x_index, val, self.numClass)
                    #print("x_index: %f; val: %f; ig: %f" % (x_index, val, ig))

                    info_gain_v.append(ig)
                    info_gain_comb.append([x_index, val])
                    val += step_size

            else:
                for x in group:
                    ig = info_gain(group, x_index, x[x_index], self.numClass)
                    #print("x_index: %f; val: %f; ig: %f" % (x_index, x[x_index], ig))

                    info_gain_v.append(ig)
                    info_gain_comb.append([x_index, x[x_index]])

        # get the split resulting maximum infomation gain
        x_index, val = info_gain_comb[np.argmax(info_gain_v)]
        # print("FINAL: x_index: %i; val: %f" % (x_index, val))
        return split(group, x_index, val), x_index, val    

In [271]:
tree = DecisionTree()
tree.fit(X_train, y_train)

print("training accuracy: ", tree.accuracy(X_train, y_train))
print("test accuracy: ", tree.accuracy(X_test, y_test))

training accuracy:  0.8709677419354839
test accuracy:  0.8420038535645472


## Random Forest

In [279]:
class RandomForest(object):
    def __init__(self, criterion='gini', max_depth=4, min_size=1, 
                 k_estimators=25, random_seed=42):
        
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_size = min_size
        self.k_estimators = k_estimators
        self.random_seed = random_seed

    # main function for training the model
    def fit(self, X, y, n_sample_size=None, d_features=None):
        
        # define n sample size if not yet defined
        if n_sample_size == None:
            self.n_sample_size = int(len(y)/10)
        else:
            self.n_sample_size = sample_size
        
        # define d feature number if not yet defined
        if d_features == None:
            self.d_features = len(X[0])
        else:
            self.d_features = d_features
        
        self.trees = []
        for i in range(self.k_estimators):
            #set random seed
            random.seed(int(self.n_sample_size))
            self.n_sample_size += 1
            rnd_index = random.sample(range(len(y)), self.n_sample_size)
            X_sampled = X[rnd_index]
            y_sampled = y[rnd_index]
            self.trees.append(self.return_DT(X_sampled, y_sampled))

        return self
    
    def return_DT(self, X_sampled, y_sampled):
        tree = DecisionTree(criterion=self.criterion, 
                            max_depth=self.max_depth, min_size=self.min_size)
        return tree.fit(X_sampled, y_sampled)
        
    #predict single x sample
    def predict(self, x):
        y_preds = [tree.predict(tree.root, x) for tree in self.trees]
        return statistics.mode(y_preds)
    
    # calculate accuracy of the model
    def accuracy(self, X, y):
        y_pred = [[self.predict(x)] for x in X]
        return (y_pred == y).sum()/len(y)

In [280]:
forest = RandomForest()
forest.fit(X_train, y_train)

print("training accuracy: ", forest.accuracy(X_train, y_train))
print("test accuracy: ", forest.accuracy(X_test, y_test))

training accuracy:  0.8577336641852771
test accuracy:  0.8670520231213873
